In [16]:
import numpy as np
import nibabel as nib
from sklearn.metrics import pairwise_distances
dcon = nib.load('HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii').get_data()

In [17]:
#restrict dcon to cerebellar cortex:
dcon_intracerebralcortical = dcon[0:59412, 0:59412]

In [19]:
dcon= dcon_intracerebralcortical

In [20]:
# Generate .dscalar file with cerebral cortical gradients:
dcon = np.tanh(dcon)
N = dcon.shape[0]
perc = np.array([np.percentile(x, 90) for x in dcon])
for i in range(dcon.shape[0]):
    print "Row %d" % i
    dcon[i, dcon[i,:] < perc[i]] = 0
dcon[dcon < 0] = 0
aff = 1 - pairwise_distances(dcon, metric = 'cosine')
np.save('cosine_affinity_intracerebralcortical.npy', aff)
aff = np.load('cosine_affinity_intracerebralcortical.npy')
import sys
sys.path.append("./mapalign")
from mapalign import embed
emb, res = embed.compute_diffusion_map(aff, alpha = 0.5)
np.save('embedding_dense_intracerebralcortical_emb.npy', emb)
np.save('embedding_dense_intracerebralcortical_res.npy', res)

res = np.load('embedding_dense_intracerebralcortical_res.npy').item()
a = [res['vectors'][:,i]/ res['vectors'][:,0] for i in range(9)]
emb = np.array(a)[1:,:].T
len(emb)
res = nib.load('hcp.tmp.lh.dscalar.nii').get_data()
cortL = np.squeeze(np.array(np.where(res != 0) [0], dtype=np.int32))
res = nib.load('hcp.tmp.rh.dscalar.nii').get_data()
cortR = np.squeeze(np.array(np.where(res != 0) [0], dtype=np.int32))
corLen = len(cortL) + len(cortR)
del res
emb = np.load('embedding_dense_intracerebralcortical_emb.npy')
emb.shape

Row 0
Row 1
Row 2
Row 3
Row 4
Row 5
Row 6
Row 7
Row 8
Row 9
Row 10
Row 11
Row 12
Row 13
Row 14
Row 15
Row 16
Row 17
Row 18
Row 19
Row 20
Row 21
Row 22
Row 23
Row 24
Row 25
Row 26
Row 27
Row 28
Row 29
Row 30
Row 31
Row 32
Row 33
Row 34
Row 35
Row 36
Row 37
Row 38
Row 39
Row 40
Row 41
Row 42
Row 43
Row 44
Row 45
Row 46
Row 47
Row 48
Row 49
Row 50
Row 51
Row 52
Row 53
Row 54
Row 55
Row 56
Row 57
Row 58
Row 59
Row 60
Row 61
Row 62
Row 63
Row 64
Row 65
Row 66
Row 67
Row 68
Row 69
Row 70
Row 71
Row 72
Row 73
Row 74
Row 75
Row 76
Row 77
Row 78
Row 79
Row 80
Row 81
Row 82
Row 83
Row 84
Row 85
Row 86
Row 87
Row 88
Row 89
Row 90
Row 91
Row 92
Row 93
Row 94
Row 95
Row 96
Row 97
Row 98
Row 99
Row 100
Row 101
Row 102
Row 103
Row 104
Row 105
Row 106
Row 107
Row 108
Row 109
Row 110
Row 111
Row 112
Row 113
Row 114
Row 115
Row 116
Row 117
Row 118
Row 119
Row 120
Row 121
Row 122
Row 123
Row 124
Row 125
Row 126
Row 127
Row 128
Row 129
Row 130
Row 131
Row 132
Row 133
Row 134
Row 135
Row 136
Row 137
Row 13

pixdim[1,2,3] should be non-zero; setting 0 dims to 1
pixdim[1,2,3] should be non-zero; setting 0 dims to 1


(59412, 8)

In [21]:
emb.shape

(59412, 8)

In [ ]:
### Since shape is 8, we need to use cerebral cortical merge8 file, generated as follows:

### Generate corticalonly_template.dscalar.nii with this:
### wb_command -cifti-create-dense-scalar corticalonly_template.dscalar.nii -left-metric L.atlasroi.32k_fs_LR.shape.gii -roi-left L.atlasroi.32k_fs_LR.shape.gii -right-metric R.atlasroi.32k_fs_LR.shape.gii -roi-right R.atlasroi.32k_fs_LR.shape.gii
### R and L L.atlasroi.32k_fs_LR.shape.gii are downloaded from https://github.com/Washington-University/Pipelines/tree/master/global/templates/91282_Greyordinates.
### If it gives an html error it is because we need to download the whole github repository, cannot download one single file with “save link as”
### Then merge the dscalar file to obtain as many dscalar maps as needed, depending on output of emb.shape below:
### wb_command -cifti-merge corticalonly_template_merge8.dscalar.nii -cifti corticalonly_template.dscalar.nii -cifti corticalonly_template.dscalar.nii -cifti corticalonly_template.dscalar.nii -cifti corticalonly_template.dscalar.nii -cifti corticalonly_template.dscalar.nii -cifti corticalonly_template.dscalar.nii -cifti corticalonly_template.dscalar.nii -cifti corticalonly_template.dscalar.nii

In [23]:
tmp = nib.load('corticalonly_template_merge8.dscalar.nii')
tmp_cifti = nib.cifti2.load('corticalonly_template_merge8.dscalar.nii')
data = tmp_cifti.get_data() * 0
mim = tmp.header.matrix[1]
for idx, bm in enumerate(mim.brain_models):
    print ((idx, bm.index_offset, bm.brain_structure))
img = nib.cifti2.Cifti2Image(emb.T, nib.cifti2.Cifti2Header(tmp.header.matrix))
img.to_filename('result_bilateralcortex.dscalar.nii')

(0, 0, 'CIFTI_STRUCTURE_CORTEX_LEFT')
(1, 29696, 'CIFTI_STRUCTURE_CORTEX_RIGHT')


In [ ]:
# 1000 group average myelin map is obtained directly from the HCP website and needs no further corrections
# (as argued in the manuscript): S1200.MyelinMap_BC_MSMAll.32k_fs_LR.dscalar.nii